In [1]:
from stock_selector import DataFetcher, DataCleaner, DataHandler, StockSelector
import numpy as np
import pandas as pd

c:\Users\hazc\anaconda3\Lib\site-packages\iFinDPy.pth


In [2]:
dataFetcher = DataFetcher()
dataHandler = DataHandler()
base_pool = pd.read_csv('codes_map.csv')
calender = dataFetcher.get_calender_data('2014-01-01', '2023-08-01', endlevel=[(1,2), (1), (1,2), (1,2)])

In [3]:
mv_data = dataFetcher.get_mkv_data(2013, 2023)
mv_data = dataHandler.mv_handler(mv_data, stock_pool=base_pool, calender=calender)
mv_rank = dataHandler.top_rank(mv_data, 0.8)
turnoverV_data = dataFetcher.get_turnoverV_data(2014, 2023)
turnoverV_data = dataHandler.turnoverV_handler(turnoverV_data, base_pool)
turnoverV_rank = dataHandler.top_rank(turnoverV_data, 0.8)
dividend_data = dataHandler.dividend_handler(dataFetcher.get_dividend_data(2011,2023), base_pool, calender)
dividend_rank = dividend_data.apply(lambda col: col[col == 1].index).apply(pd.Series).T
import numpy as np
stocks_pool = []
for i in range(115):
    common_stocks = np.intersect1d(mv_rank.iloc[:,i].dropna(), np.intersect1d(dividend_rank.iloc[:,i].dropna(), turnoverV_rank.iloc[:,i].dropna()))
    stocks_pool.append(common_stocks)
final_results = pd.DataFrame(stocks_pool, index=mv_rank.columns).T

c:\Users\hazc\Desktop\红利低波\tools\stock_selector.py:251: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  did_data['dividend_index'] = did_data.groupby('innercode')['status'].apply(rolling_check).reset_index(level=0, drop=True)


In [76]:
conn = dataFetcher.conn
query = f"""
SELECT CompanyCode, infopubldate, BasicEPS FROM LC_IncomeStatementAll
WHERE EXTRACT(YEAR FROM ENDDATE) BETWEEN 2014 AND 2015
ORDER BY CompanyCode, EndDate
"""
df = pd.read_sql(query, conn)

In [77]:
df

,companycode,infopubldate,basiceps
0,3,2014-04-24,0.53
1,3,2015-04-24,0.37
2,3,2014-08-14,0.88
3,3,2015-08-14,0.73
4,3,2015-10-23,0.41
...,...,...,...
210569,10122676,2020-02-18,NaN
210570,10141529,2020-06-10,NaN
210571,10251597,2019-09-20,NaN
210572,10255116,2019-09-25,NaN


In [61]:
len(np.intersect1d(codes_map['companycode'], df['companycode'].unique()))

4735

In [62]:
df_finance = pd.read_sql(
    """
    SELECT CompanyCode, EndDate, BasicEPS FROM LC_FIncomeStatementNew
    WHERE EXTRACT(YEAR FROM EndDate) BETWEEN 2014 AND 2023
    ORDER BY CompanyCode, EndDate
    """,
    conn)
df_nonfinance = pd.read_sql(
    """
    SELECT CompanyCode, EndDate, BasicEPS FROM LC_IncomeStatementNew
    WHERE EXTRACT(YEAR FROM EndDate) BETWEEN 2014 AND 2023
    ORDER BY CompanyCode, EndDate
    """,
    conn)
df_final = pd.concat([df_finance, df_nonfinance])



In [78]:
test = pd.read_sql("SELECT * FROM LC_IncomeStatementAll WHERE COMPANYCODE = 3 AND EXTRACT(YEAR FROM infopubldate) = 2014""", conn)

In [83]:
test[test['enddate'].dt.year == 2014]

,id,infopubldate,infosource,bulletintype,companycode,enddate,ifadjusted,ifmerged,accountingstandards,enterprisetype,...,otheroperationalcost,totaladminexpense,explorationcost,creditimpairmentp,assetimpairmentlossp,npcparentcompanyowners,npothereqinstruments,otheritemseffectingci,cicparentcompanyowners,ciothereqinstruments
3,451599296921,2014-04-24,第一季报,20,3,2014-03-31,2,1,1,13,...,None,6.062000e+09,None,None,-2.067000e+09,None,None,None,None,None
5,461261948069,2014-08-14,中期报告,20,3,2014-06-30,2,1,1,13,...,None,1.305500e+10,None,None,-5.652000e+09,None,None,None,None,None
8,467410813288,2014-10-24,第三季报,20,3,2014-09-30,2,1,1,13,...,None,2.005600e+10,None,None,-9.701000e+09,None,None,None,None,None
9,467410813835,2014-10-24,第三季报,20,3,2014-09-30,4,1,1,13,...,None,7.001000e+09,None,None,-4.049000e+09,None,None,None,None,None
